In [8]:
import pandas as pd
import numpy as np
import haversine as hs
#!pip install haversine


pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)
pd.set_option('display.precision', 0)
pd.options.display.float_format = "{:,.0f}".format

In [9]:
data = pd.read_csv("./data/kc_house_data.csv")
df_fortune_500s = pd.read_excel("./data/Fortune500LatsLongs.xlsx")

In [10]:
#Merging Cross Ref Df with Points of Intereste Df
# CPInts_With_Rank = CPInts.merge(CPInts_Xref, on="CODE", how="left", indicator=True)

# CPInts_With_Rank['tup_lat_Long'] = list(zip(CPInts_With_Rank.Y, CPInts_With_Rank.X, CPInts_With_Rank.OBJECTID, CPInts_With_Rank.NAME,CPInts_With_Rank.PointofInterest_Rank))
df_fortune_500s['tup_lat_Long_500'] = list(zip(df_fortune_500s.Lat, df_fortune_500s.Long, df_fortune_500s.Company))

# df_fortune_500s['tup_lat_Long'] = list(zip(data.lat, data.long, data.id))
data['tup_lat_Long'] = list(zip(data.lat, data.long, data.id))

In [11]:
df_Dist_From_Fortune500 = pd.DataFrame(columns = ['id', 'Company', 'Distance'])

distanceConsideredClose = 7
listofValues = []

#looping Through Main mdf
for HLat, HLong, HouseParcel in data['tup_lat_Long']:
    #print(HouseParcel)
    
    #looping through df that contains interesting places 
    for fiveH_Lat, fiveH_Long, fiveH_Company in df_fortune_500s['tup_lat_Long_500']:
        
        #getting the distance between current house and point of interest
        distance = round(hs.haversine((HLat,HLong),(fiveH_Lat,fiveH_Long), unit='mi'),2)
    
        #if point distance < 6 miles insert into a dataframe 
#         if distance < distanceConsideredClose:
        values = [HouseParcel, fiveH_Company, distance]
        listofValues.append(values)

keys = ['id', 'Company', 'Distance']
df_Dist_From_500 = pd.DataFrame(listofValues,columns = keys)


In [12]:
df_Dist_From_500.columns

Index(['id', 'Company', 'Distance'], dtype='object')

In [13]:
df_Dist_From_500["FortuneFiveBin"] = df_Dist_From_500['Distance'].apply(lambda x : "Under10" if x <10 else "Over10" if x <20 else "Over20")
df_Dist_From_500.fillna(0, inplace = True)

In [14]:
df_Dist_From_500.isnull().sum()

id                0
Company           0
Distance          0
FortuneFiveBin    0
dtype: int64

In [15]:
df_Dist_From_500.head(5)

,id,Company,Distance,FortuneFiveBin
0,7129300520,Amazon,9,Under10
1,7129300520,Costco Wholesale,10,Under10
2,7129300520,Microsoft,11,Over10
3,7129300520,Starbucks,6,Under10
4,7129300520,Paccar,10,Under10


In [16]:
df_Counts_of_Fortune500 = df_Dist_From_500[["id", "FortuneFiveBin"]].groupby(["id", "FortuneFiveBin"]).size().reset_index(name='counts')
df_Counts_of_Fortune500

,id,FortuneFiveBin,counts
0,1000102,Over10,8
1,1000102,Over20,10
2,1000102,Under10,4
3,1200019,Over10,7
4,1200019,Over20,1
...,...,...,...
58044,9895000040,Over20,1
58045,9895000040,Under10,3
58046,9900000190,Over10,5
58047,9900000190,Over20,1


In [17]:
df_Counts_of_Fortune500_pivot = pd.pivot_table(df_Counts_of_Fortune500, index='id', columns=['FortuneFiveBin'], values='counts', aggfunc='max')
df_Counts_of_Fortune500_pivot

FortuneFiveBin,Over10,Over20,Under10
id,,,
1000102,8,10,4
1200019,7,1,3
1200021,7,1,3
2800031,4,1,6
3600057,2,1,8
...,...,...,...
9842300095,4,1,6
9842300485,4,1,6
9842300540,4,1,6


# Export Files To Excel

In [ ]:
df_Counts_of_Fortune500_pivot.to_excel("df_Counts_of_Fortune500_CrossTab.xlsx")